In [ ]:
import galah
import geopandas as gpd


In [ ]:
galah.galah_config(
    email="ala.italics539@simplelogin.com",
    atlas="Australia",
)

In [ ]:
target_species = [
    "Lagarostrobos franklinii",
    "Athrotaxis cupressoides",
    "Athrotaxis selaginoides",
    "Nothofagus gunnii",
]

galah.search_taxa(taxa=target_species)


,scientificName,scientificNameAuthorship,taxonConceptID,rank,kingdom,phylum,order,family,genus,species,vernacularName,issues
0,Lagarostrobos franklinii,(Hook.f.) Quinn,https://id.biodiversity.org.au/node/apni/2892878,species,Plantae,Charophyta,Pinales,Podocarpaceae,Lagarostrobos,Lagarostrobos franklinii,Huon Pine,noIssue
1,Athrotaxis cupressoides,D.Don,https://id.biodiversity.org.au/node/apni/2909339,species,Plantae,Charophyta,Pinales,Cupressaceae,Athrotaxis,Athrotaxis cupressoides,Pencil Pine,noIssue
2,Athrotaxis selaginoides,D.Don,https://id.biodiversity.org.au/taxon/apni/5127...,species,Plantae,Charophyta,Pinales,Cupressaceae,Athrotaxis,Athrotaxis selaginoides,King Billy Pine,noIssue
3,Nothofagus gunnii,(Hook.f.) Oerst.,https://id.biodiversity.org.au/node/apni/7726150,species,Plantae,Charophyta,Fagales,Nothofagaceae,Nothofagus,Nothofagus gunnii,Tanglefoot,noIssue


In [ ]:
target_occurances = galah.atlas_occurrences(
    taxa=target_species,
)
target_occ_clean = target_occurances.dropna(
    subset=["decimalLongitude", "decimalLatitude"]
)

In [12]:
target_gdf = gpd.GeoDataFrame(
    target_occ_clean,
    geometry=gpd.points_from_xy(
        target_occ_clean["decimalLongitude"], target_occ_clean["decimalLatitude"]
    ),
    crs="EPSG:4326",
)
target_gdf = target_gdf.to_crs("EPSG:7855")

In [15]:
target_gdf.to_file("target_occurences_ala.geojson", driver="GeoJSON")